<a href="https://colab.research.google.com/github/sukyoung11/mobis/blob/main/DL_3_A_NN_backpropagation_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing 'numpy' library
import numpy as np
# Importing 'matplotlib' library to plot experimental results in form of figures
import matplotlib.pyplot as plt

In [ ]:
# 3층 신경망(입력층-은닉층-출력층) 클래스 정의
class ThreeLayersNeuralNetwork():
    def __init__(self):
        # 난수 시드 고정: 실행할 때마다 동일한 난수가 생성되어 디버깅에 유리
        np.random.seed(1)

       # 3층 신경망 구조:
        # - 입력층(Layer 0): 특성 3개
        # - 은닉층(Layer 1): 뉴런 4개
        # - 출력층(Layer 2): 뉴런 1개

        # 입력층(3) -> 은닉층(4) 가중치 초기화
        # 가중치 값의 범위: [-1, 1]
        # 형태: (3 x 4) = 입력 3개 × 은닉 4개
        self.weights_0_1 = 2 * np.random.random((3, 4)) - 1

        # 은닉층(4) -> 출력층(1) 가중치 초기화
        # 가중치 값의 범위: [-1, 1]
        # 형태: (4 x 1) = 은닉 4개 × 출력 1개
        self.weights_1_2 = 2 * np.random.random((4, 1)) - 1

        # 학습/추론 시 출력층의 값을 보관할 변수(행렬)
        self.layer_2 = np.array([])

     # 시그모이드 활성화 함수: 값을 (0,1) 범위로 정규화
    def normalizing_results(self, x):
        return 1 / (1 + np.exp(-x))

    # 시그모이드의 도함수: a * (1 - a)  (여기서 a는 시그모이드 출력)
    # 역전파 시 가중치 보정 크기(기울기)를 계산할 때 사용
    def derivative_of_sigmoid(self, x):
        return x * (1 - x)

    # 학습 후/중 추론(순전파) 실행 함수
    def run_nn(self, set_of_inputs):
        # 순전파: 입력 -> 은닉 -> 출력
        # 각 층에서 선형결합(dot) 후 시그모이드로 비선형 변환
        layer_0 = set_of_inputs  # matrix 1x3
        layer_1 = self.normalizing_results(np.dot(layer_0, self.weights_0_1))  # matrix 1x3 * matrix 3x4 = matrix 1x4
        layer_2 = self.normalizing_results(np.dot(layer_1, self.weights_1_2))  # matrix 1x4 * matrix 4x1 = matrix 1x1
        return layer_2

    # 신경망 학습(역전파 포함)
    def training_process(self, set_of_inputs_for_training, set_of_outputs_for_training, iterations):
        # 지정된 반복 횟수만큼 에폭 반복
        for i in range(iterations):
             # 1) 순전파: 입력 -> 은닉 -> 출력
            # numpy.dot으로 층의 값과 가중치를 행렬 곱해 선형결합 수행
            layer_0 = set_of_inputs_for_training  # matrix 4x3
            layer_1 = self.normalizing_results(np.dot(layer_0, self.weights_0_1))  # matrix 4x3 * matrix 3x4 = matrix 4x4
            self.layer_2 = self.normalizing_results(np.dot(layer_1, self.weights_1_2))  # matrix 4x4 * matrix 4x1 = matrix 4x1

            # 2) 출력층 오차: (정답 - 예측)
            # 형태: (4 x 1) - (4 x 1) = (4 x 1)
            layer_2_error = set_of_outputs_for_training - self.layer_2

            # 500번마다 평균 절대오차를 출력(학습 경향 관찰용)
            # (분석 전 출력 과다 방지를 원하면 이 부분을 주석 처리)
            if (i % 500) == 0:
                print('Final error after', i, 'iterations =', np.mean(np.abs(layer_2_error)))

            # 3) 출력층 델타(delta_2) 계산: 오차 × 시그모이드 도함수
            # 여기서는 원소별 곱(element-wise)을 사용('*'), dot(행렬곱)이 아님
            # 예) [[1],[1],[1],[1]] * [[2],[3],[4],[5]] = [[2],[3],[4],[5]]

            delta_2 = layer_2_error * self.derivative_of_sigmoid(self.layer_2)

            # 4) 은닉층 오차: delta_2를 출력층 가중치로 역전파
            # delta_2 · W(은닉->출력)^T  => (4x1)·(1x4)=(4x4)
            # (각 샘플의 은닉 노드별 오차 기여도)
            layer_1_error = np.dot(delta_2, self.weights_1_2.T)

            # 5) 은닉층 델타(delta_1): 은닉층 오차 × 시그모이드 도함수
            delta_1 = layer_1_error * self.derivative_of_sigmoid(layer_1)

            # 6) 가중치 업데이트(경사상승 형태; 손실을 줄이려면 -grad가 일반적이나
            #    여기 구현은 목표-예측 형태와 누적 규칙에 맞춰 += 사용)
            # 은닉->출력 가중치: (4x4)^T·(4x1) = (4x1)
            self.weights_1_2 += np.dot(layer_1.T, delta_2)
            # 입력->은닉 가중치: (4x3)^T·(4x4) = (3x4)
            self.weights_0_1 += np.dot(layer_0.T, delta_1)


In [ ]:
#ThreeLayersNeuralNetwork 클래스를 인스턴스화하여 3층 신경망을 하나 생성합니다.
#        __init__ 안에서 np.random.seed(1)로 시드를 고정했기 때문에,
#        실행할 때마다 같은 초기 가중치가 만들어집니다(디버깅/재현성 확보).
three_layers_neural_network = ThreeLayersNeuralNetwork()

#   초기(학습 전) 가중치를 확인합니다.
#        아래는 입력층(Layer 0) → 은닉층(Layer 1) 사이의 가중치 행렬을 출력.
#        모양(shape)은 (3 x 4): 입력 특성 3개 × 은닉 뉴런 4개.
#        각 값은 ∈ [-1, 1] 범위의 난수로 초기화.
print('Weights 0-1')
print(three_layers_neural_network.weights_0_1)
print()

#        은닉층(Layer 1) → 출력층(Layer 2) 사이의 가중치 행렬을 출력.
#        모양(shape)은 (4 x 1): 은닉 뉴런 4개 × 출력 뉴런 1개.
#        역시 초기 상태이므로 학습 전 무작위 값.
print('Weights 1-2')
print(three_layers_neural_network.weights_1_2)
print()

Weights 0-1
[[-0.16595599  0.44064899 -0.99977125 -0.39533485]
 [-0.70648822 -0.81532281 -0.62747958 -0.30887855]
 [-0.20646505  0.07763347 -0.16161097  0.370439  ]]

Weights 1-2
[[-0.5910955 ]
 [ 0.75623487]
 [-0.94522481]
 [ 0.34093502]]



In [ ]:
#  학습에 사용할 입력/정답(타겟) 데이터를 생성.
#  여기서는 총 4개의 샘플을 사용하며, 각 입력은 3개의 특성(feature)로 구성.
#  마지막 열이 모두 1인 이유: 이 구현에는 별도 바이어스 항이 없기 때문에,
#  "항상 1인 입력 특성"을 추가하여 바이어스 역할을 하게 합니다(일명 bias trick).
input_set_for_training = np.array([[1, 1, 1], [1, 0, 1], [0, 0, 1], [0, 1, 1]])

# 정답(타겟) 벡터 생성 : np.array([[1, 1, 0, 0]])
# 모양은 (1 x 4)이므로, .T(전치)를 적용해 (4 x 1) 열벡터로 바꿈.
# 즉, 4개 샘플 각각의 타겟이 위에서 아래로 정렬된 형태가 됩니다.
output_set_for_training = np.array([[1, 1, 0, 0]]).T

# 신경망 학습:
# 마지막 인자 5000은 반복 횟수(iterations)로, 순전파/역전파를 5000번 수행합니다.
# 내부적으로는:
#   1) 순전파: layer_0 → layer_1 → layer_2
#   2) 오차 계산: (타겟 - 예측)
#   3) 역전파: delta 계산 후, 가중치(weights_0_1, weights_1_2) 갱신
#  을 반복하여 출력이 타겟에 근접하도록 가중치를 조정.
three_layers_neural_network.training_process(input_set_for_training, output_set_for_training, 5000)

# 학습이 끝난 뒤, 최종 출력(layer_2)을 확인합니다.
# layer_2는 시그모이드 출력을 사용하므로 (0,1) 범위의 실수이며,
# 이진 분류처럼 사용하려면 보통 0.5 기준으로 0/1 판정을 합니다.

print()
print('Output results after training:')
print(three_layers_neural_network.layer_2)
print()

Final error after 0 iterations = 0.4685343254580603
Final error after 500 iterations = 0.02735966511749842
Final error after 1000 iterations = 0.018014239352682856
Final error after 1500 iterations = 0.014245380154921867
Final error after 2000 iterations = 0.012098118827883314
Final error after 2500 iterations = 0.010673901313210884
Final error after 3000 iterations = 0.009643630560125675
Final error after 3500 iterations = 0.008855140776037978
Final error after 4000 iterations = 0.008227317734746435
Final error after 4500 iterations = 0.007712523196874705

Output results after training:
[[0.99181271]
 [0.9926379 ]
 [0.00744159]
 [0.00613513]]



In [ ]:
# 단일 테스트 입력에 대한 신경망의 예측값을 출력합니다.
#  np.array([1, 0, 0])은 길이 3의 입력 벡터입니다.
#  이 구현에서는 학습 시 마지막 특성(세 번째 값)을 'bias=1'로 두는 bias trick을 썼는데,
#  여기서는 0을 넣었기 때문에 학습 때와 조건이 달라집니다(출력이 달라질 수 있음).
#  학습 조건과 동일하게 시험하려면 보통 np.array([1, 0, 1])처럼 마지막 값을 1로 둡니다.
#   run_nn(...)은 순전파(입력→은닉→출력)를 수행해 시그모이드(0~1) 범위의 값을 반환합니다.
#   이 값은 확률처럼 해석할 수 있으며, 이진 분류에선 통상 0.5를 기준으로 0/1 판정을 합니다.
#
print('Output result for testing data = ', three_layers_neural_network.run_nn(np.array([1, 0, 0])))


Output result for testing data =  [0.99619533]
